## Importing the Data


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

# Download the dataset
dataframe = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)
raw_data = dataframe.values
dataframe.head()

# The last element contains the labels
labels = raw_data[:, -1]

# The other data points are the electrocadriogram data
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=21)

#print(train_labels)

# Normalize to [0, 1]
min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
test_data = (test_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

train_data = train_data.numpy()
test_data = test_data.numpy()

#train_labels = train_labels.reshape(train_labels.shape[0], 1)
#test_labels = test_labels.reshape(test_labels.shape[0], 1)


# Creating and Training the model

In [6]:
# Claim models
import keras
import matplotlib.pyplot as plt
import time

# The input size/image
input_img = keras.Input(shape=(train_data.shape[1],))
# The size of latent space
encoding_dim = 64
## Add a simple encode layer
batchSize = 32

activationAlg = "softmax"

data = []

for count in range(20):
  startTime = time.process_time()
  # encoded = keras.layers.Dense(encoding_dim, activation='relu')(input_img)
  # You can add encode layer with a L1 activity regularizer
  encoded = keras.layers.Dense(encoding_dim, activation=activationAlg, 
                  activity_regularizer=keras.regularizers.l1(10e-5))(input_img)
  # Add decode layer
  decoded = keras.layers.Dense(train_data.shape[1], activation=activationAlg)(encoded)

  # Claim autoencoder model
  model_autoencoder = keras.Model(input_img, decoded)

  # Claim a separate encoder model
  model_encoder = keras.Model(input_img, encoded)

  # Claim a separate decoder model
  decoder_input = keras.Input(shape=(encoding_dim,))
  # Retrieve the last layer of the autoencoder model
  decoder_layer = model_autoencoder.layers[-1]
  # Create the decoder model
  model_decoder = keras.Model(decoder_input, decoder_layer(decoder_input))

  model = keras.Sequential()
  model.add(model_encoder.layers[0])
  model.add(model_encoder.layers[1])
  model.add(keras.layers.Dense(2, activation=activationAlg))
  model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  # Convert labels to categorical one-hot encoding
  one_hot_labels = keras.utils.to_categorical(train_labels, num_classes=2)

  # Train the model, iterating on the data in batches of 32 samples
  progress = model.fit(train_data, one_hot_labels, epochs=100, batch_size=batchSize)
  losses = progress.history['loss']

  y_pred_train=np.argmax(model.predict(train_data), axis=1)
  mse = mean_squared_error(train_labels,y_pred_train)
  timePassed = time.process_time() - startTime
  #data.append(str(encoding_dim) + "," + str(mse) + "," + str(timePassed))

  for i in range(100):
    data.append(activationAlg + "," + str(i) + "," + str(losses[i]))

file = open("EP-Softmax-AE.csv", "w")
#file.write("HiddenSize,Error,TrainingTime\n")
file.write("Activation,Epoch,Error\n")
for i in data:
  file.write(i + "\n")
#print(training_errors[-1][1])
file.close()

Epoch 1/100
125/125 [==============================] - 1s 1ms/step - loss: 0.6661 - accuracy: 0.5997
Epoch 2/100
125/125 [==============================] - 0s 1ms/step - loss: 0.5892 - accuracy: 0.8112
Epoch 3/100
125/125 [==============================] - 0s 1ms/step - loss: 0.5089 - accuracy: 0.9144
Epoch 4/100
125/125 [==============================] - 0s 1ms/step - loss: 0.4312 - accuracy: 0.9470
Epoch 5/100
125/125 [==============================] - 0s 1ms/step - loss: 0.3676 - accuracy: 0.9445
Epoch 6/100
125/125 [==============================] - 0s 1ms/step - loss: 0.3125 - accuracy: 0.9575
Epoch 7/100
125/125 [==============================] - 0s 1ms/step - loss: 0.2742 - accuracy: 0.9502
Epoch 8/100
125/125 [==============================] - 0s 1ms/step - loss: 0.2268 - accuracy: 0.9600
Epoch 9/100
125/125 [==============================] - 0s 1ms/step - loss: 0.2110 - accuracy: 0.9540
Epoch 10/100
125/125 [==============================] - 0s 1ms/step - loss: 0.1853 - accura

# Testing the model

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
# Overall F1 score
y_pred=np.argmax(model.predict(test_data), axis=1)
print(f1_score(test_labels, y_pred, average="macro"))
print(precision_score(test_labels, y_pred, average="macro"))
print(recall_score(test_labels, y_pred, average="macro"))
print(accuracy_score(test_labels, y_pred))

1  1.0
0.9837418861850742
0.984522316848773
0.9830357142857142
0.984
